In [2]:
# import statements 
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import GRU, Dense, Embedding 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical 



2024-03-13 17:45:53.045597: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-13 17:45:53.049120: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-13 17:45:53.094231: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-13 17:45:54.293824: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# more memory efficient use of data 
- use specific years (decade)

In [20]:
#reading our national names dataset into a value 
# dataset includes baby names in the usa, from 1880 thru 2014
national_data_set = pd.read_csv('NationalNames.csv')

# verifying our work
print(national_data_set.head())

# dropping irrelvent attributes from our dataset 
national_data_set.drop(columns='Id', inplace=True)
national_data_set.drop(columns='Gender', inplace =True)

#verifying our work 
print(national_data_set.head())

#declaring new list to store our names from the national data set, that have been parsed/cleaned
updated_national = []
just_names = []

# print(type(national_data_set["Year"][1])) # checking the data type of the year characteristic

#adding all baby names + the number of times they appeared from 1880 to our updated_national list 
for i, row in national_data_set.iterrows():
    if row["Year"] == 1880:
        just_names.append(row["Name"])
        updated_national.append({row["Name"]: row['Count']})
    else: 
        break

# commented out code for use if we have additional debugging time to add in all names from 1880 thru 2014
# for i, row in enumerate(national_data_set):
    # if row["Name"] not in updated_national:
    #     updated_national.append({row['Name']: row["Count"]})
    # elif row["Name"] in updated_national: 
    #     updated_national[row["Name"]] += row["Count"]
print(updated_national)

   Id       Name  Year Gender  Count
0   1       Mary  1880      F   7065
1   2       Anna  1880      F   2604
2   3       Emma  1880      F   2003
3   4  Elizabeth  1880      F   1939
4   5     Minnie  1880      F   1746
        Name  Year  Count
0       Mary  1880   7065
1       Anna  1880   2604
2       Emma  1880   2003
3  Elizabeth  1880   1939
4     Minnie  1880   1746
[{'Mary': 7065}, {'Anna': 2604}, {'Emma': 2003}, {'Elizabeth': 1939}, {'Minnie': 1746}, {'Margaret': 1578}, {'Ida': 1472}, {'Alice': 1414}, {'Bertha': 1320}, {'Sarah': 1288}, {'Annie': 1258}, {'Clara': 1226}, {'Ella': 1156}, {'Florence': 1063}, {'Cora': 1045}, {'Martha': 1040}, {'Laura': 1012}, {'Nellie': 995}, {'Grace': 982}, {'Carrie': 949}, {'Maude': 858}, {'Mabel': 808}, {'Bessie': 796}, {'Jennie': 793}, {'Gertrude': 787}, {'Julia': 783}, {'Hattie': 769}, {'Edith': 768}, {'Mattie': 704}, {'Rose': 700}, {'Catherine': 688}, {'Lillian': 672}, {'Ada': 652}, {'Lillie': 647}, {'Helen': 636}, {'Jessie': 635}, {'Louise

In [21]:
# defining our sample names/data 
character_names = ['gabriella', 'ryan', 'cecilia', 'kari', 'dimmer', 'allan', 'sarah', 'eliza', 'aaron', 'katnis', 'henry', 'ava', 'alice', 'bob', 'jon', 'alberto', 'marcos', 'vex', 
                   'zach', 'shahbaaz', 'fourohfour', 'pharoah', 'mia', 'katherine', 'farrow', 'ivy', 'jack', 'emily', 'bob', 'charlie', 'radhika', 
                   'quinn', 'patrick', 'eva', 'diana', 'jeffrey', 'noah', 'liam', 'puck', 'burt', 'juan', 'jose', 'chris']


# create a vocabulary based on the sample names
# vocab = sorted(set(''.join(character_names)))
vocab = sorted(set(''.join(just_names)))
char_to_idx = {char: idx for idx, char in enumerate(vocab)}
idx_to_char = {idx: char for char, idx in char_to_idx.items()}

# convert character names to sequences 
# sequences = [[char_to_idx[char] for char in name] for name in character_names]
sequences = [[char_to_idx[char] for char in name] for name in just_names]


# padding our sequences to ensure equal name length later 
max_name_length = max(len(seq) for seq in sequences)
print(type(max_name_length))
padded_sequences = pad_sequences(sequences, maxlen=max_name_length, padding='post')

# create input + target sequences 
X = padded_sequences[:, :-1]
y = padded_sequences[:, 1:]

# check the shape of our input sequence 
print(X.shape)

#check the shape of our target sequence 
print(y.shape)

# convert our target sequence into a one-hot encoding 
y_one_hot = to_categorical(y, num_classes=len(vocab))

#defining characteristics for our gru-based character generation model 
vocab_size = len(vocab)
embedding_dim = 32

# created the model
# running into issues with input_length = max_name_length-1 in our Embedding function inside the model
# return statement said it did not recognize the variable/wasn't a valid integer, but when printing the value type it returned int

model = tf.keras.Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim), 
    GRU(128, return_sequences=True), 
    Dense(vocab_size, activation='softmax')
])

# compile the model 
model.compile(optimizer='adam', loss = 'categorical_crossentropy')

# train the model 
model.fit(X, y_one_hot, epochs = 50, verbose=1)




<class 'int'>
(2000, 10)
(2000, 10)
Epoch 1/50


63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 2.9371
Epoch 2/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.6386
Epoch 3/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.5171
Epoch 4/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 1.4110
Epoch 5/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 1.3500
Epoch 6/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 1.2671
Epoch 7/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.2563
Epoch 8/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.2174
Epoch 9/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.2091
Epoch 10/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.1789
Epoch 11/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 1.1758
Epoch 12/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 1.1624
Epoch 13/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.1292
Epoch 14/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 1.1268
Epoch 15/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 1.1193
Epoch 16/50
63/63 ━━━━━━━━

In [41]:
# declare seed character 
seed = 'c'

# map our seed characters to indexes 
seed_idx = [char_to_idx[char] for char in seed]

# actually calling our model 
# finding the probabilities from the model 
# generating a name ! 
for _ in range(3):
    temperature = .95 # higher the temp -> the more random output will be 

    predicted_probs = model.predict(np.array(seed_idx).reshape(1, -1))[0]
    predicted_probs = predicted_probs[-1] # take last predicted probability from model 

    predicted_probs = np.power(predicted_probs, 1 /temperature)
    predicted_probs = predicted_probs / np.sum(predicted_probs)

    next_char = np.random.choice(list(char_to_idx.keys()), p = predicted_probs)

    seed += next_char 
    seed_idx.append(char_to_idx[next_char])

print(seed.lower())



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
cyld


In [42]:
# character age generation 

# imports 
from tensorflow.keras.layers import Input, Concatenate, Reshape 
from tensorflow.keras.models import Model

In [51]:
# create input layer for age 
input_age = Input(shape =(1,), name="input_age")
input_hair_color = Input(shape =(1,), name = "input_hair_color")


# embed our age 
embedding_age = Embedding(input_dim=X.shape[0], output_dim=embedding_dim)(input_age)
embedding_hair_color = Embedding(input_dim=X.shape[0], output_dim=embedding_dim)(input_hair_color)

# reshape the embedding 
embedding_age = Reshape((1, -1))(embedding_age)
embedding_hair_color = Reshape((1, -1))(embedding_hair_color)

common_embedding_dim = 32

# concat embeddings
concatenated_embeddings = Concatenate(axis =1)([embedding_age, embedding_hair_color])

# define our output 
gru_output = GRU(128, return_sequences=True)(concatenated_embeddings)
output = Dense(len(char_to_idx), activation='softmax')(gru_output)

# GRU based generation 
model = Model(inputs = [input_age], outputs = output)

# compile model 
model.compile(optimizer='adam', loss = 'categorical_crossentropy')

# test our model 
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_age           │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_hair_color    │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_11        │ (None, 1, 32)     │     64,000 │ input_age[0][0]   │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_12        │ (None, 1, 32)     │     64,000 │ input_hair_color… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_7 (Reshape) │ (None, 1, 32)     │          0 │ embedding_11[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_8 (Reshape) │ (None, 1, 32)     │          0 │ embedding_12[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 2, 32)     │          0 │ reshape_7[0][0],  │
│ (Concatenate)       │                   │            │ reshape_8[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_3 (GRU)         │ (None, 2, 128)    │     62,208 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 2, 51)     │      6,579 │ gru_3[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 196,787 (768.70 KB)

 Trainable params: 196,787 (768.70 KB)

 Non-trainable params: 0 (0.00 B)

In [56]:
# calling model to generate our characters ages 

num_samples = 5
sample_ages = np.random.randint(18, 30, size = num_samples)
print(sample_ages)


# appearances generator 

sample_hair_colors = ['blonde', 'brown', 'red', 'black', 'blue', 'purple']
seed = np.random.choice(just_names)
seed_idx = [char_to_idx[char] for char in seed]


for _ in range(max_name_length):
    predicted_probs = model.predict([np.array([sample_ages[_]]), np.array([seed_idx]), np.array([sample_hair_colors[_]])])[0]
    predicted_probs = predicted_probs[-1]
    next_char = np.random.choice(list(char_to_idx.keys()),p = predicted_probs)
    generated_hair_color += next_char
    seed_idx.append(char_to_idx[next_char])
sample_hair_colors.append(generated_hair_color)
print(sample_hair_colors)
    


[28 25 20 22 24]


ValueError: Invalid dtype: str192